# Risk Aggregation POC - PLTs

This notebook reads can be used to read in multiple PLTs, rollup the PLTs to an aggregate PLT, and generate statistics and EP metrics from the PLTs.

-----

### Install necessary libraries

In [ ]:
# %pip install boto3
# %pip install fsspec
# %pip install s3fs
# %pip install pyarrow
# %pip install fastparquet

### Import necessary libraries

In [ ]:
import os
import sys
sys.path.append(os.path.abspath("plttools/plt.py"))
import pandas as pd
import numpy as np
import plttools.plt
import plttools
from plttools.plt import PLT
from plttools import plt_calculator
import matplotlib.pyplot as pl

### Import data from AWS S3
AWS credentials may be provided explicitly with s3fs.S3FileSystem,
but it is more secure to exclude the credentials from the code.
Instead use the AWS CLI to `aws configure` credentials.

In [ ]:
import glob
import s3fs

s3 = s3fs.S3FileSystem(anon=False)
files = s3.glob('s3://fannie-mae-phase-3/03 Data Aggregation POC/01 Test/test_4/split/*.parquet')
plts = pd.concat([pd.read_parquet('s3://' + fp) for fp in files])

### Import local data

In [ ]:
plts = pd.read_parquet('sample_data/')

### Create PLT from imported data

In [ ]:
json_test = plts.to_dict(orient='records')
original_plt = plttools.plt.PLT(data = json_test, number_of_simulations=50000)
original_plt.plt.head(5).style

### Group account PLTs

In [ ]:
grouped_plt = plt_calculator.group_plts(original_plt)
grouped_plt.plt.head(5).style

### Rollup PLT

In [ ]:
rolled_up_plt = plt_calculator.roll_up_plts(original_plt)
rolled_up_plt.plt.head(5)

### Calculate statistics from rolled up PLT

In [ ]:
aal = rolled_up_plt.get_aal()
std = rolled_up_plt.get_standard_deviation_risk_modeler()
covvar = rolled_up_plt.get_covvar_risk_modeler()
print('AAL: ' + aal.astype(str))
print('STD: ' + str(std))
print('CovVar: ' + str(covvar))

### Calculate AEP metrics from PLT

In [ ]:
aep = plt_calculator.calculate_aep_curve(grouped_plt.plt, grouped_plt.simulations)
aep_metrics = aep.curve['Loss'][aep.curve['Loss'].index.isin([1/x for x in aep.RETURN_PERIODS])]
aep_metrics.index = np.reciprocal(aep_metrics.index)
aep_metrics = aep_metrics.rename_axis("Return Period").sort_index(axis=0, ascending=False)

# Return AEP results
aep_metrics.to_frame().style.format("{:,.0f}")

### Calculate OEP metrics from PLT

In [ ]:
oep = plt_calculator.calculate_oep_curve(grouped_plt.plt, grouped_plt.simulations)
oep_metrics = oep.curve['Loss'][oep.curve['Loss'].index.isin([1/x for x in oep.RETURN_PERIODS])]
oep_metrics.index = np.reciprocal(oep_metrics.index)
oep_metrics = oep_metrics.rename_axis("Return Period").sort_index(axis=0, ascending=False)

# Return OEP results
oep_metrics.to_frame().style.format("{:,.0f}")